In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.decomposition import PCA

# Método de la Potencia

En muchas aplicaciones del mundo real de la ciencia y la ingeniería, se requiere encontrar numéricamente el valor Eigen más grande o dominante y el Eigenvector correspondiente. Existen diferentes como el método de la potencia que sigue un enfoque iterativo y conforma un algoritmo simple.

El siguiente algoritmo busca desarrollar los pasos necesario para el método de la potencia y así después pues ser implementado como otro método alternativo a los demás que se tienen en el proyecto.

Dada una matriz $A$ diagonalizable, el algoritmo de la potencia genera un número $\lambda$ que es el eigenvalor más grande (en valor asoluto) de $A$, y un vector no nulo $v$, que es el eigenvector correspondiente a $\lambda$, es decir, son tales que $Av = \lambda v$.

El método de la potencia comienza con un vector $b_0$, que puede ser un vector aleatoio, o bien una aproximación al eigenvector dominante. La relación de recurrencia que describe al método es:
$$b_{k+1}= \frac{Ab_k}{\|Ab_k\|}$$
Por lo que, en cada iteración, el vector $b_k$ es multiplicado por la matriz $A$ y luego normalizado.

Buscamos una sucesión $(b_k)$ que converja a un eigenvector Para asegurar la convergencia, necesitan cumplirse las siguientes condiciones:

- $A$ tiene un eigenvalor estrictamente mayor en magnitud respecto a sus otros eigenvalores
- El vector de inicio $b_0$ tiene una componente distinta de cero en la dirección de un eigenvector asociado con el eigenvalor dominante.

Además, bajo las dos supoosiciones anteriores, la sucesión $(\mu_k)$ definida por:
$$\mu_k = \frac{b_k^TAb_k}{b_k^Tb_k}$$
converge al eigenvalor dominante. 


In [2]:
df = pd.read_csv('../../data/nndb_flat.csv', encoding = "UTF-8")

In [3]:
df.drop(df.columns[df.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)
df = df.drop(columns=['ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName','ScientificName'])

In [4]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(df)

In [5]:
X

array([[ 2.89623357, -1.01174721,  4.44128945, ..., -0.64991809,
        -0.41055694, -0.55991833],
       [ 2.89623357, -1.01174721,  4.44128945, ..., -0.65484222,
        -0.41055694, -0.57183012],
       [ 3.83495634, -1.06577576,  5.59915265, ..., -0.75332487,
        -0.44590424, -0.58374191],
       ...,
       [ 0.25127886, -1.0923161 , -0.67108312, ..., -0.72870421,
        -0.42116113, -0.53013886],
       [-0.80552224,  0.43375349, -0.58284096, ...,  0.57126681,
         0.52261173, -0.28892514],
       [-0.81142615,  0.78446513, -0.63956806, ...,  0.11824661,
         0.14793037, -0.28892514]])

In [6]:
# Matriz de Covarianzas
C = np.dot(X.T, X)/X.shape[0]

Definimos la función que implementa el método de la potencia para obtener el eigenvalor dominante y su correspondiente eigenvector:

In [7]:
def power_iteration(A, num_simulations: int):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm
    
    #Obtenemos el eigenvalor correspondiente a b_k con el cociente de Rayleigh
    m_k = (b_k.T@A@b_k)/(b_k.T@b_k)
    
    #Devolvemos el mayor eigenvalor y su correspondiente eigenvector
    return m_k,b_k
    

In [8]:
m_1,b_1 = power_iteration(C,500)

In [9]:
#eigenvalor dominante
m_1

5.449285764311568

In [10]:
#eigenvector correspondiente al eigenvalor dominante
b_1

array([0.15781363, 0.1406199 , 0.03300812, 0.16968451, 0.07632275,
       0.18156988, 0.13351884, 0.31566282, 0.17798456, 0.08763937,
       0.13712201, 0.28410233, 0.33777895, 0.34132464, 0.27245318,
       0.16811197, 0.18080591, 0.2998569 , 0.24134847, 0.09356749,
       0.19940258, 0.0923186 , 0.24355123])

Podemos comparar los valores obtenidos con el primer eigenvalor y eigenvector:

In [11]:
evalues, evectors = np.linalg.eig(C)

In [12]:
evalues[0]

5.449285764311579

In [13]:
evectors.T[0]

array([-0.15781363, -0.1406199 , -0.03300812, -0.16968451, -0.07632275,
       -0.18156988, -0.13351884, -0.31566282, -0.17798456, -0.08763937,
       -0.13712201, -0.28410233, -0.33777895, -0.34132464, -0.27245318,
       -0.16811197, -0.18080591, -0.2998569 , -0.24134847, -0.09356749,
       -0.19940258, -0.0923186 , -0.24355123])

In [14]:
np.allclose(m_1,evalues[0])

True

In [15]:
np.allclose(np.abs(b_1),np.abs(evectors.T[0]))

True

## Deflation

Con lo anterior, hemos obtenido el eigenvalor de mayor magnitud y su correspondiente eigenvalor. Sin embargo, para hacer el PCA necesitamos los demás eigenvalores. Es aquí donde entra el método de *deflation*. Este consiste en volver a aplicar el método a una matriz actualizada:
$$A_{k+1}= A_k - b_kb_k^TA_kb_kb_k^T$$

Apliquemos dicha transformación a la matriz de covarianzas $C$ para obtener el segundo eigenvalor de $C$ y su correspondiente eigenvector:

In [16]:
C_def = C- np.outer(b_1,b_1)@C@np.outer(b_1,b_1)

In [17]:
m_2,b_2 = power_iteration(C_def,100)

In [18]:
m_2

2.6184583526663694

In [19]:
b_2

array([ 0.27344866, -0.34339677,  0.11167035,  0.44341644,  0.35876913,
        0.25773287, -0.23647025, -0.02112889, -0.35504461,  0.03852455,
        0.10637166,  0.09709328, -0.08480073, -0.07347102,  0.0751503 ,
        0.10517259, -0.21266898,  0.09381195,  0.10336119, -0.08878313,
       -0.08744763, -0.23932243, -0.17779772])

Podemos corroborar que estos valores aproximan el segundo eigenvalor y su correspondiente eigenvector

In [20]:
evalues[1]

2.6184583526663796

In [21]:
evectors.T[1]

array([-0.27344866,  0.34339677, -0.11167035, -0.44341644, -0.35876913,
       -0.25773287,  0.23647025,  0.02112889,  0.35504461, -0.03852455,
       -0.10637166, -0.09709328,  0.08480073,  0.07347102, -0.0751503 ,
       -0.10517259,  0.21266898, -0.09381195, -0.10336119,  0.08878313,
        0.08744763,  0.23932243,  0.17779772])

Así, creamos una función que combine el método de la potencia y *deflation*:


In [22]:
def power_deflation(A,iter):
    #numero de columnas
    n = A.shape[1]
    # Inicializamos arrays de ceros
    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n,n))
    #Hago una copia de la matriz original
    A_def = A.copy()
    #Iteramos tantas veces como columnas de la matriz
    for i in range(n):
        #Aplicamos el método de la potencia
        m_def,b_def = power_iteration(A_def,iter)
        #Actualizamos los arrays de eigen valores y vectores
        eigenvalues[i] = m_def
        eigenvectors[:,i]= b_def
        # Matriz actualizada
        A_def = A_def - np.outer(b_def,b_def)@A_def@np.outer(b_def,b_def)
    return eigenvalues, eigenvectors

In [23]:
evalues_pow, evectors_pow = power_deflation(C,50)

Podemos notar que (salvo por el orden decreciente en que nosotros obtenemos los valores), se trata de una buena aproximación:

In [24]:
# Lo que obtenemos
evalues_pow

array([5.44928576e+00, 2.61845835e+00, 2.03189746e+00, 1.87912807e+00,
       1.63567543e+00, 1.14036709e+00, 1.06087106e+00, 9.26189477e-01,
       8.61952843e-01, 8.24826963e-01, 7.31232602e-01, 5.96894949e-01,
       5.08479245e-01, 4.69648488e-01, 4.07987975e-01, 3.35026344e-01,
       3.31768786e-01, 3.22018315e-01, 2.56187042e-01, 2.38154652e-01,
       2.11160240e-01, 1.59093679e-01, 3.79524723e-03])

In [25]:
#Lo que esperamos (ordenado de forma decreciente)
evalues = np.sort(evalues)[::-1]
evalues

array([5.44928576e+00, 2.61845835e+00, 2.03189759e+00, 1.87913144e+00,
       1.63567140e+00, 1.14037066e+00, 1.06086727e+00, 9.26253795e-01,
       8.62018168e-01, 8.24686441e-01, 7.31232556e-01, 5.96894964e-01,
       5.08653057e-01, 4.69461661e-01, 4.07986316e-01, 3.37934003e-01,
       3.29568690e-01, 3.21238326e-01, 2.56202403e-01, 2.38139303e-01,
       2.11158912e-01, 1.59093679e-01, 3.79524723e-03])

## PCA

In [29]:
def PCA_from_potencia(X):
    prop = 0 #Proporción de varianza explicada
    comp = 1 
    cur_var = 0
    comp_vecs = np.zeros([X.shape[1], X.shape[1]])
    
    # convertir a array
    A = np.array(X)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)
    
    #Calculamos la matriz de covarianzas
    cov = np.dot(X.T, X)/X.shape[0]
    
    #Aplicamos el método de la potencia
    evalues_pow, evectors_pow = power_deflation(cov,50)
    
    # La varianza explicada
    varianza_explicada = evalues_pow/np.sum(evalues_pow)
    
    # Los datos transformados (componentes principales)
    Z = datos_centrados@evectors_pow
    
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 80%
    n = X.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.8
    num_componentes = conteo.sum() + 1
    
    return evalues_pow[:num_componentes], evectors_pow[:num_componentes], Z[:,:num_componentes], varianza_explicada[:num_componentes] 
    

In [30]:
eigenvalues, componentes, Z, var_explicada = PCA_from_potencia(X)

In [31]:
eigenvalues

array([5.44928576, 2.61845835, 2.0318866 , 1.8791433 , 1.63567143,
       1.14037024, 1.06086173, 0.9259546 , 0.86107886, 0.82600963])

In [32]:
componentes

array([[ 0.15781363, -0.27343437,  0.46244017,  0.04813368,  0.29358612,
         0.13270953,  0.06666283,  0.07717095,  0.06127852, -0.13225757,
        -0.13979036,  0.11710477, -0.05756254, -0.06707062,  0.03933277,
         0.06923014, -0.09501888, -0.04719535,  0.08541941,  0.06795423,
        -0.12889147, -0.03809124, -0.67801123],
       [ 0.1406199 ,  0.34340328,  0.2109066 , -0.31306212, -0.01305824,
         0.14145138,  0.00590418,  0.12625714,  0.14083793,  0.00741331,
        -0.30697748, -0.12972777, -0.31117387,  0.08437973, -0.13236093,
         0.13908246,  0.00315365,  0.49869381,  0.06022338,  0.22226724,
        -0.2536767 , -0.10876264,  0.1789299 ],
       [ 0.03300812, -0.11165384,  0.53425612,  0.02174598,  0.39445097,
        -0.11659471,  0.04534104, -0.0560572 , -0.04948317, -0.21370583,
        -0.05339109,  0.20191276,  0.10427672, -0.10183696,  0.03322662,
        -0.08718438,  0.22193676, -0.01895494, -0.09335814, -0.04616235,
         0.15878048,  0.0367

In [33]:
Z

array([[-1.12177585e+00, -1.18213797e+00,  3.67098479e+00, ...,
        -5.98229058e-01, -3.07553933e-01, -1.50894797e+00],
       [-1.11468691e+00, -1.18405956e+00,  3.67145779e+00, ...,
        -5.96024041e-01, -3.16549964e-01, -1.51673776e+00],
       [-9.94919411e-01, -1.57343402e+00,  4.70874213e+00, ...,
        -5.87928030e-01, -3.22616373e-01, -1.85540140e+00],
       ...,
       [-7.67670698e-01, -3.26768624e+00, -9.68622209e-01, ...,
         8.34093206e-01,  4.17901563e-01,  2.06187652e+00],
       [ 3.55897094e-01,  6.78466168e-01,  9.97247069e-01, ...,
         6.62557619e-01, -5.47727873e-02,  1.06179821e-01],
       [-8.66889802e-01,  1.19845293e+00, -1.97560729e-01, ...,
        -2.49720128e-01, -6.41314855e-02,  4.44507507e-04]])

In [34]:
var_explicada

array([0.2369236 , 0.11384512, 0.0883422 , 0.08170124, 0.07111559,
       0.04958092, 0.04612406, 0.04025858, 0.03743792, 0.03591318])

## Referencias
- Wikipedia [Power iteration](https://en.wikipedia.org/w/index.php?title=Power_iteration&oldid=957783806) (last visited May 29, 2020)
- Mackey, Lester. (2008). Deflation Methods for Sparse PCA. Advances in Neural Information Processing Systems 21 - Proceedings of the 2008 Conference. 21. 1017-1024. 
- Power Method Algorithm for Finding Dominant Eigen Value and Eigen Vector. (n.d.). Retrieved May 23, 2020, from https://www.codesansar.com/numerical-methods/power-method-algorithm-for-finding-dominant-eigen-value-and-eigen-vector.htm

- Fox, J., Chalmers, P., Monette, G., & Sanchez, G. (2020, April 14). powerMethod: Power Method for Eigenvectors in matlib: Matrix Functions for Teaching and Learning Linear Algebra and Multivariate Statistics. Retrieved from https://rdrr.io/cran/matlib/man/powerMethod.html

- Dan, D. J. (n.d.). dianejdan/Power-Method-PCA. Retrieved May 27, 2020, from https://github.com/dianejdan/Power-Method-PCA/blob/master/power-pca.py